In [22]:
# import libs 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.optim import lr_scheduler
import numpy as np 
import torchvision 
from torchvision import datasets,transforms,models 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt 
import time 
import os 
import copy 
import glob 

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [24]:
# Transforms
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5 , 0.5 , 0.5 ],[0.5 , 0.5 , 0.5 ]),
])

In [25]:
# set directory paths 
base_dir = "basedata"
training = "training"
validationString = "validation"

# training ,validation locations 
training_path = os.path.join(base_dir,training)
test_path = os.path.join(base_dir,"testing")

In [26]:
# DataLoader 
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(training_path , transform = transformer),
    batch_size = 12 , shuffle= True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path , transform = transformer),
    batch_size = 12 , shuffle= True
)

In [27]:
# get classes 
classes = os.listdir(training_path)

In [28]:

# Set HyperParameters 
num_epochs = 3 
batch_size = 4 
learning_rate = 0.001
# Tune this later based on performance 

In [29]:
class TransferLearning(nn.Module):
    def __init__(self,num_classes = 41):
        super(TransferLearning,self).__init__()
        self.model = models.resnet18(pretrained = True)
        input = self.model.fc.in_features

        # setting final layer
        self.model.fc = nn.Linear(input,num_classes)
    def forward(self,input):
        output = self.model(input)
        return output

In [30]:
model = TransferLearning(num_classes=41).to(device)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.001)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i , (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)


        # Forward pass 
        outputs = model(images)
        loss = criterion(outputs,labels)

        # Backward pass 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    # if (i+1) % 2 == 0:
    print('Epoch {} , Loss : {}'.format(epoch,loss.item())) 

print('Finished Training')

Epoch 0 , Loss : 3.708106279373169
Epoch 1 , Loss : 3.911158561706543
Epoch 2 , Loss : 4.251499652862549
Finished Training


In [35]:
# For testing 
with torch.no_grad():
    n_correct = 0 
    n_samples = 0 
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]

    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _ , predicted = torch.max(outputs , 1)
        n_samples +=labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]

            if (label == pred):
                n_class_correct[label] +=1
            n_class_samples[label]+=1
    
    accuracy = 100.0* n_correct/n_samples 
    print('Accuracy  : {} % '.format(accuracy))

    for i in range(len(classes)):
        if n_class_samples[i] == 0:
            print('Didnt get the sample in test ')
        else:
            acc = 100*n_class_correct[i]/n_class_samples[i]
            print('Accuracy of {}  : {} %'.format(classes[i],acc))

Accuracy  : 81.05590062111801 % 
Accuracy of 023  : 100.0 %
Accuracy of 004  : 100.0 %
Accuracy of 033  : 100.0 %
Accuracy of 037  : 75.0 %
Accuracy of 010  : 0.0 %
Accuracy of 013  : 100.0 %
Accuracy of 035  : 100.0 %
Accuracy of 039  : 75.0 %
Accuracy of 001  : 100.0 %
Accuracy of 025  : 100.0 %
Accuracy of 041  : 100.0 %
Accuracy of 029  : 66.66666666666667 %
Accuracy of 016  : 100.0 %
Accuracy of 009  : 100.0 %
Accuracy of 026  : 100.0 %
Accuracy of 028  : 100.0 %
Accuracy of 022  : 100.0 %
Accuracy of 006  : 50.0 %
Accuracy of 027  : 100.0 %
Accuracy of 030  : 75.0 %
Accuracy of 002  : 0.0 %
Accuracy of 034  : 100.0 %
Accuracy of 021  : 100.0 %
Accuracy of 024  : 0.0 %
Accuracy of 005  : 100.0 %
Accuracy of 031  : 100.0 %
Accuracy of 007  : 50.0 %
Accuracy of 008  : 100.0 %
Accuracy of 017  : 66.66666666666667 %
Accuracy of 014  : 100.0 %
Accuracy of 020  : 75.0 %
Accuracy of 036  : 100.0 %
Accuracy of 038  : 100.0 %
Accuracy of 015  : 50.0 %
Accuracy of 011  : 100.0 %
Accuracy of